In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import os

2025-06-27 13:47:21.974091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751006842.027325  101050 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751006842.043341  101050 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751006842.152365  101050 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751006842.152418  101050 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751006842.152426  101050 computation_placer.cc:177] computation placer alr

In [4]:
# Load CSV
df = pd.read_csv("multi_label_dataset.csv")

df.head()

,filename,label
0,image_001.jpg,acne;oily
1,image_002.jpg,acne
2,image_003.jpg,acne
3,image_004.jpg,acne
4,image_005.jpg,acne


In [5]:
# Split label strings into lists
df['label_list'] = df['label'].apply(lambda x: x.split(';'))

df.head()

,filename,label,label_list
0,image_001.jpg,acne;oily,"[acne, oily]"
1,image_002.jpg,acne,[acne]
2,image_003.jpg,acne,[acne]
3,image_004.jpg,acne,[acne]
4,image_005.jpg,acne,[acne]


In [6]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['label_list'])

# Optional: convert back to DataFrame for visibility
label_df = pd.DataFrame(Y, columns=mlb.classes_)
print(label_df.head())

   acne  darkspot  dry  normal  oily  other  wrinkle
0     1         0    0       0     1      0        0
1     1         0    0       0     0      0        0
2     1         0    0       0     0      0        0
3     1         0    0       0     0      0        0
4     1         0    0       0     0      0        0


In [7]:
# Store full image paths
df['filepath'] = df['filename'].apply(lambda x: os.path.join('dataset', x))

# Split into train/val
X_train, X_val, y_train, y_val = train_test_split(df['filepath'], Y, test_size=0.2, random_state=42)

df.head()

,filename,label,label_list,filepath
0,image_001.jpg,acne;oily,"[acne, oily]",dataset/image_001.jpg
1,image_002.jpg,acne,[acne],dataset/image_002.jpg
2,image_003.jpg,acne,[acne],dataset/image_003.jpg
3,image_004.jpg,acne,[acne],dataset/image_004.jpg
4,image_005.jpg,acne,[acne],dataset/image_005.jpg


In [8]:
# Store full image paths
df['filepath'] = df['filename'].apply(lambda x: os.path.join('dataset', x))

# Split into train/val
X_train, X_val, y_train, y_val = train_test_split(df['filepath'], Y, test_size=0.2, random_state=42)

## Data Preparation

In [9]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Function to load and preprocess image
def load_image(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    return img, label

# Build tf.data.Dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train.values, y_train))
train_ds = train_ds.map(load_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val.values, y_val))
val_ds = val_ds.map(load_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

I0000 00:00:1751006845.946083  101050 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2279 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


## Model

In [10]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid')  # Multi-label output
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])  # For multi-label, you can also add AUC

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,855 (9.24 MB)

 Trainable params: 164,871 (644.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Training

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

In [13]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/50


I0000 00:00:1751006849.874099  101156 service.cc:152] XLA service 0x7fb840002260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751006849.874158  101156 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-06-27 13:47:29.972310: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751006850.546315  101156 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-06-27 13:47:31.599332: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4377_0', 464 bytes spill stores, 1372 bytes spill loads

2025-06-27 13:47:31.658544: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_

 2/71 ━━━━━━━━━━━━━━━━━━━━ 7s 106ms/step - accuracy: 0.1094 - loss: 0.7849 

I0000 00:00:1751006859.267051  101156 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


70/71 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4572 - loss: 0.4334

2025-06-27 13:47:42.717577: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4377', 12 bytes spill stores, 12 bytes spill loads

2025-06-27 13:47:42.982314: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4377', 8 bytes spill stores, 8 bytes spill loads

2025-06-27 13:47:43.189646: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4377_0', 184 bytes spill stores, 520 bytes spill loads

2025-06-27 13:47:43.360894: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4377', 8 bytes spill stores, 8 bytes spill loads

2025-06-27 13:47:43.401241: I external/l

71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.4586 - loss: 0.4323

2025-06-27 13:47:52.474126: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1188', 4 bytes spill stores, 4 bytes spill loads

2025-06-27 13:47:52.653350: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1181', 8 bytes spill stores, 8 bytes spill loads

2025-06-27 13:47:52.747222: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1181', 220 bytes spill stores, 220 bytes spill loads

2025-06-27 13:47:52.987751: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1181_0', 480 bytes spill stores, 1372 bytes spill loads

2025-06-27 13:47:53.259925: I externa

71/71 ━━━━━━━━━━━━━━━━━━━━ 31s 276ms/step - accuracy: 0.4601 - loss: 0.4313 - val_accuracy: 0.6536 - val_loss: 0.2898 - learning_rate: 0.0010
Epoch 2/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.6629 - loss: 0.2797 - val_accuracy: 0.6856 - val_loss: 0.2714 - learning_rate: 0.0010
Epoch 3/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.7122 - loss: 0.2532 - val_accuracy: 0.6892 - val_loss: 0.2612 - learning_rate: 0.0010
Epoch 4/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.7446 - loss: 0.2260 - val_accuracy: 0.7158 - val_loss: 0.2505 - learning_rate: 0.0010
Epoch 5/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.7748 - loss: 0.2093 - val_accuracy: 0.7194 - val_loss: 0.2468 - learning_rate: 0.0010
Epoch 6/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.7900 - loss: 0.1966 - val_accuracy: 0.7140 - val_loss: 0.2472 - learning_rate: 0.0010
Epoch 7/50
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.7963 - loss: 0.1817 - val_accuracy: 0.

## Evaluation

In [14]:
import numpy as np
from sklearn.metrics import classification_report, multilabel_confusion_matrix

# Predict on validation set
y_pred_prob = model.predict(val_ds)
y_pred = (y_pred_prob > 0.5).astype(int)  # thresholding for multi-label

# Ground truth labels
y_true = np.concatenate([y for _, y in val_ds], axis=0)

# Print classification report per label
print(classification_report(y_true, y_pred, target_names=mlb.classes_))

# Optional: confusion matrix per label
conf_matrix = multilabel_confusion_matrix(y_true, y_pred)

for i, label in enumerate(mlb.classes_):
    print(f"\n🔍 Confusion Matrix for '{label}':\n{conf_matrix[i]}")


18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 174ms/step
              precision    recall  f1-score   support

        acne       0.90      0.88      0.89       232
    darkspot       0.17      0.02      0.04        48
         dry       0.85      0.59      0.70       103
      normal       0.78      0.76      0.77       143
        oily       0.72      0.50      0.59       102
       other       0.00      0.00      0.00         2
     wrinkle       1.00      0.02      0.05        43

   micro avg       0.83      0.63      0.72       673
   macro avg       0.63      0.40      0.43       673
weighted avg       0.79      0.63      0.67       673
 samples avg       0.73      0.69      0.70       673


🔍 Confusion Matrix for 'acne':
[[308  23]
 [ 28 204]]

🔍 Confusion Matrix for 'darkspot':
[[510   5]
 [ 47   1]]

🔍 Confusion Matrix for 'dry':
[[449  11]
 [ 42  61]]

🔍 Confusion Matrix for 'normal':
[[390  30]
 [ 34 109]]

🔍 Confusion Matrix for 'oily':
[[441  20]
 [ 51  51]]

🔍 Confusion Matrix for 'oth

2025-06-27 13:49:23.519596: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/home/ajusd/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ajusd/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Testing

In [15]:
def load_image_test(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    img = tf.expand_dims(img, axis=0)  # 👈 Add batch dimension
    return img

img_acne = load_image_test('test_detection/acne_test.jpeg')  # Make sure the filename has the correct extension
y_test = model.predict(img_acne)

# Get top 3 predictions (no thresholding)
top_indices = y_test[0].argsort()[-3:][::-1]  # top 3 indices in descending order
top_labels = [(mlb.classes_[i], y_test[0][i]) for i in top_indices]

# Print predictions
for label, prob in top_labels:
    print(f"{label}: {prob * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
normal: 50.40%
acne: 22.28%
darkspot: 9.87%


In [16]:
img_oily = load_image_test('test_detection/oily_test.jpeg')  # Make sure the filename has the correct extension
y_test = model.predict(img_oily)

# Get top 3 predictions (no thresholding)
top_indices = y_test[0].argsort()[-3:][::-1]  # top 3 indices in descending order
top_labels = [(mlb.classes_[i], y_test[0][i]) for i in top_indices]

# Print predictions
for label, prob in top_labels:
    print(f"{label}: {prob * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
acne: 55.81%
wrinkle: 23.62%
oily: 18.89%


In [17]:
img_oily = load_image_test('test_detection/oily_test2.jpg')  # Make sure the filename has the correct extension
y_test = model.predict(img_oily)

# Get top 3 predictions (no thresholding)
top_indices = y_test[0].argsort()[-3:][::-1]  # top 3 indices in descending order
top_labels = [(mlb.classes_[i], y_test[0][i]) for i in top_indices]

# Print predictions
for label, prob in top_labels:
    print(f"{label}: {prob * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
oily: 76.96%
wrinkle: 22.05%
acne: 9.31%


In [18]:
img_oily = load_image_test('test_detection/image_test.jpeg')  # Make sure the filename has the correct extension
y_test = model.predict(img_oily)

# Get top 3 predictions (no thresholding)
top_indices = y_test[0].argsort()[-3:][::-1]  # top 3 indices in descending order
top_labels = [(mlb.classes_[i], y_test[0][i]) for i in top_indices]

# Print predictions
for label, prob in top_labels:
    print(f"{label}: {prob * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
normal: 62.10%
dry: 11.66%
darkspot: 5.08%


In [19]:
img_wrinkle = load_image_test('test_detection/wrinkle_test.jpeg')  # Make sure the filename has the correct extension
y_test = model.predict(img_wrinkle)

# Get top 3 predictions (no thresholding)
top_indices = y_test[0].argsort()[-3:][::-1]  # top 3 indices in descending order
top_labels = [(mlb.classes_[i], y_test[0][i]) for i in top_indices]

# Print predictions
for label, prob in top_labels:
    print(f"{label}: {prob * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
dry: 66.80%
acne: 29.14%
oily: 26.55%


## Save Model

In [24]:
model.save('classify.keras')  # Save in new Keras 3 format

## Loading

In [25]:
loaded_model = tf.keras.models.load_model('classify.keras')

In [26]:
img_wrinkle = load_image_test('test_detection/wrinkle_test.jpeg')  # Make sure the filename has the correct extension
y_test = loaded_model.predict(img_wrinkle)

# Get top 3 predictions (no thresholding)
top_indices = y_test[0].argsort()[-3:][::-1]  # top 3 indices in descending order
top_labels = [(mlb.classes_[i], y_test[0][i]) for i in top_indices]

# Print predictions
for label, prob in top_labels:
    print(f"{label}: {prob * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
dry: 66.80%
acne: 29.14%
oily: 26.55%


In [28]:
img_wrinkle = load_image_test('test_detection/wrinkle2_test.jpg')  # Make sure the filename has the correct extension
y_test = loaded_model.predict(img_wrinkle)

# Get top 3 predictions (no thresholding)
top_indices = y_test[0].argsort()[-3:][::-1]  # top 3 indices in descending order
top_labels = [(mlb.classes_[i], y_test[0][i]) for i in top_indices]

# Print predictions
for label, prob in top_labels:
    print(f"{label}: {prob * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
dry: 65.53%
wrinkle: 24.63%
normal: 16.20%


In [30]:
mlb.classes_

array(['acne', 'darkspot', 'dry', 'normal', 'oily', 'other', 'wrinkle'],
      dtype=object)

In [31]:
model.save("classify2.keras", save_format="keras")

In [32]:
model.save("classify.h5")  # Save in HDF5 format